In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from utils.robot_function import detect_frontier, sparse_point_cloud,change_color
import copy
import scipy.ndimage
from utils.astar import grid_path
from sklearn.cluster import DBSCAN
from collections import Counter
from utils.tools import extract_door,create_segmentation, componenets_connection_check, get_local_map
from utils.tools import room, create_room_topo
from utils.tools import map_predictor
from utils.topo_path_plan import calculate_path_on_topo


class agent:
    def __init__(self,map,init_pose,global_map,node_path = None,dir_use = False) -> None:
        self.map = map
        self.global_map = global_map
        self.predicted_map = None
        self.pose = init_pose

        #for robot exploration
        self.total_frontier = np.array([],dtype=int).reshape(-1,2)
        self.clustered_frontier = np.array([],dtype=int).reshape(-1,2)
        self.topo_map_planning = calculate_path_on_topo(global_map,sample_rate=10,sparse_dis=8,vis=True,dis_mat_path=node_path,dir_use=dir_use)

        self.lambda_ = 0.05


        self.last_frontier = [-100,-100]

        self.h_rad = 10
        self.h_gain = 2
        self.info_gain_range = 5


    def obtain_information_gain(self,frontier):
        #计算信息增益
        #基于预测地图的信息增益计算
        predict_info_gain_map = np.logical_and(self.predicted_map == 255,self.map == 100) #预测为可通行的未知区域
        info_gain_list = []
        for point in frontier:
            local_map = get_local_map(predict_info_gain_map, point, self.info_gain_range,defailt_value=0)
            info_gain = np.sum(local_map)
            info_gain_list.append(info_gain)
        # print("info gain list",info_gain_list)
        return np.array(info_gain_list)
    
    def choose_goal_pre(self,room_mask = None):
        if room_mask is None:
            room_mask = np.zeros(self.map.shape)
        
        #only keep the fontier with prior mask smaller than 1
        frontier_prior = room_mask[self.total_frontier[:,0],self.total_frontier[:,1]]

        frontier = self.total_frontier

        if len(frontier) == 0:
            frontier = self.total_frontier

        #using NBV to make decision in these frontiers
        frontier_poses = frontier
        dis_cost = np.zeros(len(frontier_poses))
        for i in range(len(frontier_poses)):
            dis_cost[i] = self.topo_map_planning.path_plan([np.array(self.pose), np.array(frontier_poses[i])])

        #NBV part
        frontier_info_gain = self.obtain_information_gain(frontier_poses)
        frontier_info_gain[dis_cost < self.h_rad] *= self.h_gain

        value_list = frontier_prior

        util_func = np.exp(self.lambda_*value_list) * dis_cost 

        min_index = np.argmin(util_func)


        choose_frontier = copy.deepcopy(frontier_poses[min_index])
        self.last_frontier = copy.deepcopy(frontier_poses[min_index])


        # np.delete(self.total_frontier, max_index, axis=0)
        return choose_frontier

    def update_frontier(self):
        delete_index = []
        for index, frontier in enumerate(self.total_frontier):
            if self.map[frontier[0],frontier[1]] != 100:
                delete_index.append(index)

        self.total_frontier = np.delete(self.total_frontier, delete_index, axis = 0)

        current_frontier = detect_frontier(self.map) 
        total_frontier = np.vstack((self.total_frontier, current_frontier))
        ds_size = 5
        
        self.total_frontier = total_frontier

        clustered_frontier = sparse_point_cloud(total_frontier, ds_size)

        clustered_frontier = self.frontier_cluster(clustered_frontier,cluser_eps = 8,cluster_min_samples = 1).reshape((-1,2))
        self.clustered_frontier = clustered_frontier

    def frontier_cluster(self,frontiers, cluser_eps=0.7, cluster_min_samples=1):
        # input: frontier; DBSCAN eps; DBSCAN min samples
        # output: clustered frontier
        if len(frontiers) == 0:
            return frontiers
        dbscan = DBSCAN(eps=cluser_eps, min_samples=cluster_min_samples)
        labels = dbscan.fit_predict(frontiers)
        label_counts = Counter(labels)
        clustered_frontier = []
        for label_key in label_counts.keys():
            if label_key == -1:
                continue
            label_index = labels==label_key
            # frontier_center = np.sum(frontiers[label_index,:],axis=0)/label_counts[label_key]
            # clustered_frontier.append(frontier_center)
            clustered_frontier.append(frontiers[label_index,:][0])

        return np.array(clustered_frontier)
    
    def navigate_to_point(self,point):
        if self.global_map[point[0], point[1]] != 255:
            print("target in obs, failed in navigation")
            return [self.pose, point]
        now_global_map = self.map
        distance_map = scipy.ndimage.distance_transform_edt(now_global_map != 0) 
        calculate_grid_path = grid_path(now_global_map,distance_map,(self.pose[0],self.pose[1]), [(point[0],point[1])],free_flag=255)
        calculate_grid_path.get_path()

        return calculate_grid_path.foundPath[0]

class sim_env:
    def __init__(self,global_map, init_pose,laser_range,vis = False) -> None:
        #以ROW 为x， 以COL 为y
        self.map = global_map #0:free, 255: unknown, 100:obstacle

        self.vis_mask = np.zeros(global_map.shape).astype(bool)
        self.pose = init_pose #[row, col]
        self.pose_list = []
        self.pose_list.append(copy.deepcopy(self.pose))

        self.laser_range = laser_range #int

        self.gen_laser_point()

        self.agent_map = None
        self.vis = vis
    
    def update_pose(self,new_pose):
        self.pose = copy.deepcopy(new_pose)
        if self.vis:
            self.pose_list.append(copy.deepcopy(self.pose))
        
    def gen_laser_point(self,min_angle=0,max_angl=360,resolution=1):
        laser_point = []

        for angle in range(min_angle, max_angl, resolution):
            tmp_point = []
            angle_rad = np.radians(angle)
            x1 = int(self.laser_range * np.cos(angle_rad))
            y1 = int(self.laser_range * np.sin(angle_rad))
            
            #生成点
            distance = max(abs(x1), abs(y1))
            if distance==0:
                return False
            step_x = x1  / distance
            step_y = y1 / distance

            for i in range(int(distance) + 1):
                x = int( i * step_x)
                y = int( i * step_y)
                tmp_point.append((x,y))
            laser_point.append(tmp_point)
        self.laser_point = laser_point
    
    def update_vis(self):
        # for a certain point, update the visiablity
        for laser in self.laser_point:
            for dxdy in laser:
                next_x = self.pose[0] + dxdy[0]
                next_y = self.pose[1] + dxdy[1]
                self.vis_mask[next_x,next_y] = 1
                if self.map[next_x,next_y] != 255: #find not free space in GT map
                    break
    
    def update_agent_map(self):
        #update map for the agent
        self.agent_map = copy.deepcopy(self.map)
        self.agent_map[~self.vis_mask] = 100
    
    def vis_map_points(self,fig,ax, map,points,point_frame = 'uv',scale = 10):
        #point:n*2 array
        #point frame should choose from uv rc
        if point_frame=='uv':
            points = points.reshape((-1,2))
        elif point_frame =='rc':
            points = points.reshape((-1,2))
            points = np.flip(points,axis=1)
        else:
            print("Error point frame")
            return
        ax.axis('equal')
        ax.set_xlabel('X')
        ax.set_ylabel('Y')

        ax.scatter(points[:,0], points[:,1], scale,color='red', marker='o')
        ax.imshow(map, cmap='gray')
        return fig,ax
        

/home/master/anaconda3/envs/PreExp/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
np.random.seed(1)

#start sim exploration
def sim_exp(now_file, now_start_point, vis=False, lambda_= 0.1):
    now_png = os.path.join(dataset_path, now_file)
    input_image = cv2.imread(now_png, cv2.IMREAD_GRAYSCALE)

    grid_size = 0.2
    
    now_sim = sim_env(input_image,now_start_point,12//grid_size,True) #激光雷达范围

    model_path = f"./trained_model/best_fpunet_128.pth"
    unet_map_predictor = map_predictor(model_path)

    now_sim.update_vis()
    now_sim.update_agent_map()

    node_path = "./dataset/node_dis_file/" + now_file + ".npz"

    now_agent = agent(now_sim.agent_map,now_sim.pose,now_sim.map,node_path=node_path,dir_use=True)
    now_agent.lambda_ = lambda_


    now_agent.update_frontier()
    total_step = 1000
    debug_flag = False
    for i in range(total_step):
        # print(f"step: {i}/{total_step}")
        if len(now_agent.total_frontier) == 0:
            # print("no frontier")
            break
        #get the room topo    
        predicted_map = unet_map_predictor.predict_map(now_agent.map, now_agent.clustered_frontier,debug_flag)
        now_agent.predicted_map = predicted_map
        
        door_point, room_point = extract_door(predicted_map,debug=debug_flag)
        seg_map = create_segmentation(predicted_map,door_point,room_point,debug_flag) #create room and door segmentation
        now_topo = create_room_topo(now_agent.map,predicted_map,seg_map)
        #visualize the room topo
        # now_topo.vis_topo(robot_pose=now_agent.pose,save_path=f"./tmp/{i}.png")
        target_mask_map = now_topo.selec_next_room(now_agent.pose) #None or a int array

        next_goal = now_agent.choose_goal_pre(target_mask_map) # value for next ponit

        
        now_path = now_agent.navigate_to_point(next_goal)
        if len(now_path) == 0:
            print("failed to find path")
        
        for next_pose in now_path:
            now_agent.pose = copy.deepcopy([next_pose[0],next_pose[1]])
            now_sim.update_pose([next_pose[0],next_pose[1]])
            now_sim.update_vis()
            #if the target is already visable, change another goal
            #not a frontier
            local_vis_map = now_sim.vis_mask[next_goal[0]-1 :next_goal[0]+2 ,next_goal[1]-1 : next_goal[1]+2]
            #0 in local vis map
            if not 0 in local_vis_map:
                break
            
        now_sim.update_agent_map()
        now_agent.map = now_sim.agent_map
        now_agent.update_frontier()
    
    if vis:
        # print(f"step: {i}/{total_step}")
        new_map = now_sim.map

        fig, ax = plt.subplots()
        total_pose_list = np.array(now_sim.pose_list)
        show_mask = np.logical_or(now_sim.vis_mask, new_map != 255) #show the free space with vis_mask == 1 or map occupied space
        alpha_channel = np.where(show_mask, 0, 0.5)  

        
        ax.imshow(new_map,cmap='gray')
        ax.imshow(show_mask, cmap='gray', alpha=alpha_channel)

        num_points = len(total_pose_list)
        colors = plt.cm.cool(np.linspace(0, 1, num_points)) 
        
        for i in range(num_points - 1):
            ax.plot(
                total_pose_list[i:i+2, 1],  
                total_pose_list[i:i+2, 0],  
                color=colors[i],            
                linewidth=1                 
            )
        
        #start_point
        ax.scatter(now_start_point[1],now_start_point[0],100,color='green', marker='*',zorder = 5)
        ax.scatter(total_pose_list[-1,1],total_pose_list[-1,0],80,color=np.array([147,112,219])/255, marker='^',zorder = 5)
        ax.axis('off')
        

        #save fig
        exp_name ="pre_fig"
        save_dir = f"./dataset/exp_data/{exp_name}/"
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        fig.savefig(os.path.join(save_dir,now_file), dpi=300,pad_inches=0.0,bbox_inches = 'tight')
        # fig.show()
        plt.close(fig)
    
    #calculate path length
    total_pose_list = np.array(now_sim.pose_list)

    differences = total_pose_list[1:] - total_pose_list[:-1]

    squared_differences = np.sum(np.square(differences), axis=1)

    path_length = np.sum(np.sqrt(squared_differences))

    return path_length*grid_size


In [3]:
#load the star point
import csv
#load the file
scene_name_list = []
start_point_list = []
file_path = os.path.join("./dataset/exp_data","noise_start.csv")
with open(file_path, 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        scene_name_list.append(row[0])
        start_point_list.append(eval(row[1]))
        

In [4]:
import os


vis_flag = False
dataset_path = f"./dataset/test_noised"

#list all file in dataset_path
file_list = os.listdir(dataset_path)
file_start_list = []
for file in file_list:
    file_start_list.append(start_point_list[scene_name_list.index(file)])

count = 0
add_newflag = False


for now_file, now_start_point in zip(file_list, file_start_list):
 
    
    path_length = sim_exp(now_file, now_start_point, vis=vis_flag, lambda_=0.08)


    print(f"Completed tasks {count}: {now_file}")
    print(f"path length {path_length}")

            

500
